In [20]:
import os
import json

import openai

from youtube import download_captions_and_meta
from summarize import chunk_a_text

In [21]:
GPT_MODEL_NAME = "gpt-3.5-turbo"
GPT_MODEL_MAX_TOKENS = 2049
openai.api_key = os.environ['OPENAI']

In [22]:
# m = download_captions_and_meta("https://www.youtube.com/watch?v=z3ohWzO9-TY")
with open('/mnt/storage/newsboy/z3ohWzO9-TY-meta.json') as f:
    m = json.load(f)

In [86]:
text = ' '.join(x['text'] for x in m['captions'])
chunks = list(chunk_a_text(text, 4000))

In [88]:
def summarize_with_gpt(text, model_name=GPT_MODEL_NAME):
    assert text
    assert isinstance(text, str)

    resp = openai.ChatCompletion.create(
      model=model_name,
      messages=[
            FIRST_TOKEN,
            {"role": "user", "content": f"{text}\n\nWrite a summary for the text above"}
      ]
    )
    return resp['choices'][0]['message']['content']



In [89]:
# Write a summary for the text above

In [111]:
m['captions_track']

'a.ru'

In [90]:
len(chunks)

27

In [96]:
t = chunks[7]
t

'зался я причем  только не так я давно понял что это тот  звонкий с которого как бы ну я не  фанател с него просто для меня было я  взял какого-то артиста фотографа для  меня это было Волков Ну типа да потому  что я жил на Красногвардейской там Кады  еще не было там где я жил там не был  МКАДа То есть у меня из-за окон  кукурузные поля были Вот и короче тогда  вот это было и потом когда вот ты  двигаешься оно уже как-то  Вау типа мы справились я понял В общем  это с первых Ага Слушай я вот не могу не  спросить Это наверное вообще я такая  одна из глобальных важных тем вот я на  тебя всегда смотрю и знаешь как  воспринимаю что ты работал там с  артистами всю свою жизнь у вас огромное  количество артистов было есть твой  прямой партнёр большая звезда и вот я  просто понимаю что когда знаешь как  компании появляется какой-то сотрудник  звезда это уже определённая такая не то  чтобы проблема но это всегда сложность  потому что с одной стороны если этот  человек круто работает Ты хочешь его

In [100]:
FIRST_TOKEN = {"role": "system", "content": "You are a helpful assistant that helps a user understand a video."}
resp = openai.ChatCompletion.create(
  model=GPT_MODEL_NAME,
  messages=[
        FIRST_TOKEN,
        {"role": "user", "content": f"{t}\n\nWrite summary for the text of the video. Use language of the text"}
  ]
)
s = resp['choices'][0]['message']['content']
s

"The video discusses how to work with celebrities and high-performing individuals. The speaker explains that it's important to differentiate between the sane and insane individuals, as well as those who are so self-absorbed that they are impossible to work with. The video also highlights the importance of finding common ground and establishing a professional relationship with high-performing individuals while recognizing the potential for competition. Furthermore, the speaker discusses the difficulties of working with individuals who possess a sense of entitlement and how to deal with them professionally."

In [116]:
FIRST_TOKEN = {"role": "system", "content": ""}
resp = openai.ChatCompletion.create(
  model=GPT_MODEL_NAME,
  messages=[
        FIRST_TOKEN,
        {"role": "user", "content": f"{t}\n\nWrite summary for the text in Russian"}
  ]
)
s = resp['choices'][0]['message']['content']
s

'Текст о том, как работать с артистами и звездами, которые осознают свою ценность. Автор разделяет людей на более и менее адекватных и утверждает, что с теми, кто оторван от реальности, нельзя работать. Также обсуждается конкуренция и необходимость находить точки соприкосновения.'

In [117]:
FIRST_TOKEN = {"role": "system", "content": ""}
resp = openai.ChatCompletion.create(
  model=GPT_MODEL_NAME,
  messages=[
        FIRST_TOKEN,
        {"role": "user", "content": f"{t}\n\nКраткое содержание приведенного выше текста"}
  ]
)
s = resp['choices'][0]['message']['content']
s

'В тексте автор обсуждает сложности работы с артистами и людьми, которые имеют определенную звездную ценность. Он различает людей по уровню адекватности и утверждает, что некоторых невозможно удержать или работать с ними. Ключевыми моментами, которые следует учитывать, при работе с этими людьми, являются конкуренция и наличие аудитории. Он также отмечает небрежный и неправильный подход молодежи к артистам, который приводит к гипертрофированию и нереальным ожиданиям.'

In [104]:
FIRST_TOKEN = {"role": "system", "content": ""}
resp = openai.ChatCompletion.create(
  model=GPT_MODEL_NAME,
  messages=[
        FIRST_TOKEN,
        {"role": "user", "content": f"{t}\n\nВыдели главные тезисы"}
  ]
)
s = resp['choices'][0]['message']['content']
s

'1. Работа с артистами - сложная задача из-за наличия у них определенной аудитории и осознания своей ценности.\n\n2. Люди могут быть более или менее адекватными, и с менее адекватными людьми сложнее работать.\n\n3. Невозможно найти точку соприкосновения с человеком, который настолько самовлюблен в себя, что начинает охеревать от себя.\n\n4. В среде молодежи сейчас много американской тенденции, когда люди не отвечают за свои слова.\n\n5. Важно учитывать конкуренцию и находить общую точку соприкосновения при работе с артистами.\n\n6. Главная задача - не удержать артиста, а найти общий язык и перевести на другой формат работы.'

# Crosslingual prompting and dynamic chunking

In [29]:
import os
import deepl
from transformers import GPT2TokenizerFast
from polyglot.detect import Detector
import openai

openai.api_key = os.environ['OPENAI']

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

t = """Quantum computing is a type of computing that relies on the principles of quantum mechanics to process information. In classical computing, information is processed using bits that can be in either a 0 or 1 state. In quantum computing, information is processed using quantum bits or qubits, which can be in a 0 or 1 state, or in a superposition of both states at the same time.

This property of superposition allows quantum computers to perform certain calculations much faster than classical computers. Additionally, quantum computers can take advantage of another property of quantum mechanics called entanglement, which allows qubits to be connected in a way that allows them to work together in a highly correlated way.

Quantum computing is still in the early stages of development, and current quantum computers are not yet powerful enough to outperform classical computers for most tasks. However, researchers are continuing to make progress in developing more powerful quantum computers, and it is possible that in the future, quantum computing could revolutionize fields such as cryptography, materials science, and drug discovery."""

In [35]:
deepl2iso = {'EN-GB': 'en', 'EN-US': 'en', 
             'ID': 'nn', 'NB': 'nn', 
             'PT-BR': 'pt', 'PT-PT': 'pt'}

In [13]:
translator = deepl.Translator(os.environ['DEEPL'])

sizes = {'en': len(tokenizer(t)['input_ids'])}
for lang in translator.get_target_languages():
    result = translator.translate_text(t, target_lang=lang)
    d = Detector(result.text)
    print(lang.code, d.languages[0].code)
    print(len(tokenizer(result.text)['input_ids']))
    print()
    sizes[d.languages[0].code] = len(tokenizer(result.text)['input_ids'])
    
sizes




In [37]:
# en_cmd = "Write a short summary for the text above"
en_cmd = "Summarize the text"

cmds = {'en': en_cmd}
# for lang in ['ru']: #translator.get_target_languages():
for lang in translator.get_target_languages():
    result = translator.translate_text(en_cmd, target_lang=lang)
    L = deepl2iso.get(lang.code, lang.code).lower()
    print(L, result.text)
    print()
    cmds[L] = result.text
cmds

bg Обобщаване на текста

cs Shrnutí textu

da Sammenfatning af teksten

de Fassen Sie den Text zusammen

el Συνοψίστε το κείμενο

en Summarize the text

en Summarize the text

es Resumir el texto

et Võtke tekst kokku

fi Tiivistä teksti

fr Résumez le texte

hu A szöveg összefoglalása

nn Meringkas teks

it Riassumere il testo

ja 本文を要約する

ko 텍스트 요약

lt Apibendrinti tekstą

lv Apkopojiet tekstu

nn Oppsummer teksten

nl De tekst samenvatten

pl Podsumuj tekst

pt Resumir o texto

pt Resumir o texto

ro Rezumați textul

ru Резюмируйте текст

sk Zhrňte text

sl Povzemanje besedila

sv Sammanfattning av texten

tr Metni özetleyin

uk Підсумуйте текст

zh 归纳文本



{'en': 'Summarize the text',
 'bg': 'Обобщаване на текста',
 'cs': 'Shrnutí textu',
 'da': 'Sammenfatning af teksten',
 'de': 'Fassen Sie den Text zusammen',
 'el': 'Συνοψίστε το κείμενο',
 'es': 'Resumir el texto',
 'et': 'Võtke tekst kokku',
 'fi': 'Tiivistä teksti',
 'fr': 'Résumez le texte',
 'hu': 'A szöveg összefoglalása',
 'nn': 'Oppsummer teksten',
 'it': 'Riassumere il testo',
 'ja': '本文を要約する',
 'ko': '텍스트 요약',
 'lt': 'Apibendrinti tekstą',
 'lv': 'Apkopojiet tekstu',
 'nl': 'De tekst samenvatten',
 'pl': 'Podsumuj tekst',
 'pt': 'Resumir o texto',
 'ro': 'Rezumați textul',
 'ru': 'Резюмируйте текст',
 'sk': 'Zhrňte text',
 'sl': 'Povzemanje besedila',
 'sv': 'Sammanfattning av texten',
 'tr': 'Metni özetleyin',
 'uk': 'Підсумуйте текст',
 'zh': '归纳文本'}

In [31]:
t = 'поприветствуем Нового гостя четвертый  разбор  Как зовут Данил зовут Сколько лет раз  два года хорош Какой у тебя ниша Чем  занимаешься мы занимаемся производством  раздвижных антимоскитных систем мы  представительства У нас есть свои мы не  производители именно комплектующих  именно самой системы Это у нас  занимается поставщики Мы работаем только  с розницей вас это что такое сеточки на  окна которые складываются раскладываются  их в общем не ремонтируешь каждый год и  так далее основной клиент у нас это  частные дома Вот мы остальные все  отбрасываем квартиры так далее только  дома только дома частные Понятно в  Самаре работаете Самарской области  продаем по России Понятно А можешь  объяснить без моделей вообще то есть У  кого что закупаете как деньги рождаются  мы занимаемся именно сборкой и продажей  У нас есть производители которые берут  металлургических заводах комплектующие  для изделий вот мы сами в принципе  собираем по размерам и устанавливаем  клиенту Конечно вы у завода закупаете  Или вы просто платите работать закупили  перепродали клиентам установили и  зарабатываем маржу как на перепродаже  так и установке Да все верно Понятно чем  самом производстве то что мы из  комплектующих собираем готовые изделия  пример комплектующие такие рамы всякие  сетки вы все собираете это ваша наценка  потом перепродаете за эту наценка из  установки правильно Окей хорошо продаете  и устанавливаете по всей России только  Самарской области только Самарского  продаем по всей устанавливаем только  Самарской области сто процентов выручки  то Сколько занимает продажа Сколько  занимает продажи простановка в Самарской  области продаж  70%. то есть выручки это продажа по  России продажи по основной области 70%.  а то есть где-то вот так короче это  Самара А вот так это Россия А сколько  чистой прибыли приносят это направление  но они будут по 30 процентов  А как так рентабельность установкой  такая же как на продажу у нас установка  получается сотрудники  почему зачем сотрудники работают  Ну во-первых так проще считать во-вторых  мы не путаемся в-третьих  то что мы можем продавать изделия Как  так и со своей установкой там и просто  продажа Вот такая схема сок сотрудников  пять пять сотрудников ты сам это все  придумал но мы зашли в проект это мы  взяли представительство Я просто говорю  почему не знаю мой Николай биполярка  и получается мы начали как представители  сами работали до сентября этого года мне  потом это все надоело я не захотел  заниматься розницей потому что я  посмотрел по стоимости что-то там в  итоге получается по чистой прибыли Я  думаю такая история интересная Я решил  продавать В итоге сентябрьской вот эта  вся история началась уже который после  20 числа у меня слетели покупатели я  такой типа ну ладно пусть постоит А как  ты попал в эту тему вот такой желтый  занимались  ч'


resp = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": ""},
        {"role": "user", "content": f"{t}\n\nРезюмируйте текст"}
  ]
)
resp['choices'][0]['message']['content']

'Разговор с Новым гостем был о его бизнесе, который занимается производством и продажей раздвижных антимоскитных систем для частных домов. Он закупает комплектующие у производителей и собирает изделия, которые затем продает и устанавливает по всей России. Основной клиент - частные дома в Самарской области, где продажи составляют 70% выручки. Он также заявил, что рентабельность установки и продажи у них примерно одинакова, и у них есть пять сотрудников. Гость начал заниматься этим бизнесом как представитель, но в итоге решил продавать самостоятельно.'